# Data Challenge 3, Costly Conversion
## Liz Norred

Summary: General recommendation is to change the pricing model to $59 for the product. While conversion decreased by 0.4 percent, revenue per user increased by 18 percent. Variable distributions of converted users were roughly equivalent, though it was noted that users tend to purchase much more in the later half of the month. Users on mobile devices converted 0.5 percent more in the test case, and converting users in general use mobile devices more often.


### Import Libraries and Data

In [1]:
import numpy as np
import pandas as pd
from pandas_profiling import ProfileReport

import seaborn as sns

We run into problems right away when the datetime values can't be inferred. 3% of the dataset contains these values. Roughly the same number of test cases (3.3% for low and 3.2% for high) contain these NaT values, so let's just exclude them.

In [7]:
test = pd.read_csv('/home/liz/IDS-Data-Challenges/test_results.csv', index_col='user_id',header = 0, parse_dates=['timestamp'],infer_datetime_format=True)
time_user= pd.to_datetime(test.timestamp.values, format="%Y-%m-%d %H:%M:%S", errors='coerce')
test['time_user']=time_user
test=test.drop(columns='timestamp')
test=test.dropna()
print(test.shape)

(306529, 7)


In [ ]:
ProfileReport(test)

In [8]:
users = pd.read_csv('/home/liz/IDS-Data-Challenges/user_table.csv', index_col='user_id',header = 0)
print(users.shape)


(275616, 4)


In [ ]:
ProfileReport(users)

All values in the users table are in the US, so this column can be excluded. The test column appears to be a one-hot encoded version of either 39 or 59 dollar pricing. The datetime values are in user time, not standard UTC time, so if we want to look at them in aggregate over time, this will need to be converted-- TimezoneFinder can do this for us without an online lookup, since we have the lat/long values. So let's join these tables (knowing that since the tables are not the same size, we'll have some NaN values in the geographical data.

In [9]:
test=test.join(users)
test.drop(columns=['country'])

,source,device,operative_system,test,price,converted,time_user,city,lat,long
user_id,,,,,,,,,,
604839,ads_facebook,mobile,iOS,0,39,0,2015-05-08 03:38:34,Buffalo,42.89,-78.86
624057,seo-google,mobile,android,0,39,0,2015-05-10 21:08:46,Lakeville,44.68,-93.24
317970,ads-bing,mobile,android,0,39,0,2015-04-04 15:01:23,Parma,41.38,-81.73
685636,direct_traffic,mobile,iOS,1,59,0,2015-05-07 07:26:01,Fayetteville,35.07,-78.90
820854,ads_facebook,web,mac,0,39,0,2015-05-24 11:04:40,Fishers,39.95,-86.02
...,...,...,...,...,...,...,...,...,...,...
17427,ads_facebook,web,windows,0,39,0,2015-04-11 09:29:15,NaN,NaN,NaN
687787,direct_traffic,web,windows,0,39,0,2015-03-16 23:31:55,Bell Gardens,33.97,-118.15
618863,ads-google,web,mac,0,39,0,2015-04-11 01:35:19,West Seneca,42.84,-78.75


### Looking at the two different conditions

In [10]:
lowprice=test[test['test'] == 0]
highprice=test[test['test'] == 1]

In [9]:
ProfileReport(lowprice)

Number of variables,12
Number of observations,196073
Total Missing (%),4.3%
Total size in memory,18.0 MiB
Average record size in memory,96.0 B
Numeric,3
Categorical,5
Boolean,2
Date,1
Text (Unique),0
Rejected,1


In [10]:
ProfileReport(highprice)

Number of variables,12
Number of observations,110456
Total Missing (%),4.3%
Total size in memory,10.1 MiB
Average record size in memory,96.0 B
Numeric,3
Categorical,5
Boolean,2
Date,1
Text (Unique),0
Rejected,1


For the high price, 1715(1.6%) converted and 108741 didn't convert out of 110456. For the low price, 3905(2.0%) converted and 192168 didn't convert out of 196073. This suggests that the lower price performed better overall in conversions than the higher price, but values are not yet normalized by category. A higher proportion of users in the low price used mobile devices( high, 59.4%; low, 57.8%). This % change was also visible in the operative_system category. When I ran this initially with the faulty timestamps included, the low price category acquired 1% more users from direct traffic. This disappears in the corrected data, suggesting this error is associated with web traffic. There initally do not appear to be dramatic differences in the test samples-- but there are some mistakes in the price data! Since we don't know which category is correct between price and test, we should delete these samples. Deleting them does not noticably change the conversion numbers. 

In [11]:
# Delete users that have incompatible price/test data
lowprice= lowprice[lowprice.price != 59]
highprice= highprice[highprice.price != 39]

Now that we've cleaned the data and removed problematic samples, let's look at the revenue generated from these samples

In [20]:
low_conv=lowprice[lowprice['converted'] == 1]
high_conv=highprice[highprice['converted'] == 1]

low_sum= sum(low_conv['price'])
high_sum= sum(high_conv['price'])
low_sum1= sum(low_conv['price'])/196073
high_sum1= sum(high_conv['price'])/110456


print('The revenue for $39 case is', low_sum)
print('The revenue for $59 case is', high_sum)
print('The average revenue per user for $39 case is', low_sum1)
print('The average revenue per user for $59 case is', high_sum1)

print('The ratio of test:control is', high_sum1/low_sum1)

The revenue for $39 case is 152139
The revenue for $59 case is 101126
The average revenue per user for $39 case is 0.7759303932718936
The average revenue per user for $59 case is 0.9155319765336424
The ratio of test:control is 1.1799150857760394


It seems like the high price case generated more revenue, despite an overall reduction in the conversion rate. The test case outperformed the control case per user by about 18%. So this is the option we should choose. The next thing we can do is look at the conversions that happened in each case, and see common characteristics.

In [21]:
ProfileReport(low_conv)

In [22]:
ProfileReport(high_conv)

Converting users generally tend to:
    1. Make purchases at the end of the month
    2. Have roughly the same distribution of most common cities
    3. Have more users on mobile (59.5% control, 59.9% test)
    4. Have roughly the same distribution of operating system choices
    5. Have roughly the same distribution of network sources

It may be helpful to reorganize these users into state or regional groups. I can tell there are subtle differences in the lat/long data, but this may be easier to see organized into territories. I can also convert the local signup time to   UTC values.